In [1]:
import numpy as np
import matplotlib.pyplot as plt

import fiona
import pyproj
from shapely.geometry import shape, MultiPolygon, Polygon
from shapely.geos import TopologicalError
from descartes.patch import PolygonPatch


In [2]:
precincts = fiona.open("../shapefiles/police_precincts/nypp.shp")

In [3]:
tracts = fiona.open("../shapefiles/census_tracts/nyct2010.shp")

In [4]:
def convert_geometry_to_gps(geometry, orig_proj):
    original = pyproj.Proj(orig_proj, preserve_units=True)
    destination = pyproj.Proj(init='epsg:4326')

    if geometry['type'] == 'Polygon':
        return shape({
            'type': 'Polygon',
            'coordinates': [
                [
                    pyproj.transform(original, destination, x, y)
                    for (x,y) in path
                ]
                for path in geometry['coordinates']
            ]
        })
    elif geometry['type'] == 'MultiPolygon':
        return shape({
            'type': 'MultiPolygon',
            'coordinates': [
                [
                    [
                        pyproj.transform(original, destination, x, y)
                        for (x,y) in path
                    ]
                    for path in polygon
                ]
                for polygon in geometry['coordinates']
            ]
        })
    else:
        return None


In [5]:
tract_dict = {}
precinct_dict = {}

for tract in tracts:
    tract_dict[tract['id']] = {
        'shape': convert_geometry_to_gps(
            tract['geometry'],
            tracts.crs
        ),
        'intersections': []
    }
    

for precinct in precincts:
    if precinct['id'] == '35':
        precinct_geom = convert_geometry_to_gps(
            {
                'type': 'MultiPolygon',
                'coordinates': [ precinct['geometry']['coordinates'][260] ]
            },
            precincts.crs
        )
    elif precinct['id'] == '70':
        precinct_geom = convert_geometry_to_gps({
            'type': 'MultiPolygon',
            'coordinates': [
                precinct['geometry']['coordinates'][0],
                precinct['geometry']['coordinates'][103]
            ]},
            precincts.crs
        )
    else:
        precinct_geom = convert_geometry_to_gps(
            precinct['geometry'],
            precincts.crs
        )
    
    precinct_dict[precinct['id']] = {
        'shape': precinct_geom,
        'intersections': []
    }

for tract in tracts:
    if (int(tract['id']) % 50) == 0:
        print('Processing tract {0}'.format(tract['id']))
    for precinct in precincts:
        if tract_dict[tract['id']]['shape'].intersects(precinct_dict[precinct['id']]['shape']):
            tract_dict[tract['id']]['intersections'].append(precinct['id'])
            precinct_dict[precinct['id']]['intersections'].append(tract['id'])

Processing tract 0
Processing tract 50
Processing tract 100
Processing tract 150
Processing tract 200
Processing tract 250
Processing tract 300
Processing tract 350
Processing tract 400
Processing tract 450
Processing tract 500
Processing tract 550
Processing tract 600
Processing tract 650
Processing tract 700
Processing tract 750
Processing tract 800
Processing tract 850
Processing tract 900
Processing tract 950
Processing tract 1000
Processing tract 1050
Processing tract 1100
Processing tract 1150
Processing tract 1200
Processing tract 1250
Processing tract 1300
Processing tract 1350
Processing tract 1400
Processing tract 1450
Processing tract 1500
Processing tract 1550
Processing tract 1600
Processing tract 1650
Processing tract 1700
Processing tract 1750
Processing tract 1800
Processing tract 1850
Processing tract 1900
Processing tract 1950
Processing tract 2000
Processing tract 2050
Processing tract 2100
Processing tract 2150


In [6]:
print('Mean number of tracts per precinct: {0}'.format(
    np.mean([len(v['intersections']) for k, v in precinct_dict.items()])
))
print('Mean number of precincts per tract: {0}'.format(
    np.mean([len(v['intersections']) for k, v in tract_dict.items()])
))

Mean number of tracts per precinct: 42.467532467532465
Mean number of precincts per tract: 1.5096952908587258
